### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
%pip install lightfm
%pip install implicit

     |████████████████████████████████| 302 kB 846 kB/s eta 0:00:01
  Using cached numpy-1.19.1-cp37-cp37m-manylinux2010_x86_64.whl (14.5 MB)
  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl (26.1 MB)
  Using cached requests-2.22.0-py2.py3-none-any.whl (57 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached urllib3-1.25.10-py2.py3-none-any.whl (127 kB)
  Using cached certifi-2020.6.20-py2.py3-none-any.whl (156 kB)
  Created wheel for lightfm: filename=lightfm-1.15-cp37-cp37m-linux_x86_64.whl size=705693 sha256=dc808e29c69762829a4c63dc5520debe54d68fadb53ab634c2f20f518bce5dd3
  Stored in directory: /home/jupyter/.cache/pip/wheels/f0/cd/a5/b07914aa223c05ed61880d4c59f64a7febf117dbd2c2cbcf49
Successfully built lightfm
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you 

In [2]:
%cd hse_recsys2020/HomeWorks

/home/jupyter/work/resources/hse_recsys2020/HomeWorks


In [3]:
import sys
from subprocess import Popen
from subprocess import PIPE

stdout, stderr = Popen(["cat", "ml-1m/users.dat", "."], stdout=PIPE, stderr=PIPE).communicate()
print(stdout.decode().split("\n")[:10])
print(stderr.decode(), file=sys.stderr)

del stdout
del stderr

['1::F::1::10::48067', '2::M::56::16::70072', '3::M::25::15::55117', '4::M::45::7::02460', '5::M::25::20::55455', '6::F::50::9::55117', '7::M::35::1::06810', '8::M::25::12::11413', '9::M::25::17::61614', '10::F::35::1::95370']


cat: .: Is a directory



In [4]:
%pwd

'/home/jupyter/work/resources/hse_recsys2020/HomeWorks'

In [5]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [6]:
import os
import shutil
import zipfile
import urllib.request

filename = 'ml-1m.zip'
urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-1m.zip', 'ml-1m.zip')

('ml-1m.zip', <http.client.HTTPMessage at 0x7fce0d60f290>)

In [7]:
import zipfile
from tqdm import tqdm

fname = './ml-1m.zip'
path = './'

with zipfile.ZipFile(fname, 'r') as zf:
    for entry in tqdm(zf.infolist(), desc='Extracting '):
        try:
            zf.extract(entry, path)
        except zipfile.error as e:
            pass


Extracting : 100%|██████████| 5/5 [00:03<00:00,  1.30it/s]
/py-env/platform-env/lib/python3.7/site-packages/ml_kernel/kernel.py:405: UserWarning: The following variables cannot be serialized: zf
  warnings.warn(message)


In [8]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')

In [9]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [10]:
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [11]:
len(ratings)

1000209

Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [12]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [13]:
implicit_ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
3,1,3408,4,978300275
4,1,2355,5,978824291
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368
10,1,595,5,978824268
11,1,938,4,978301752
12,1,2398,4,978302281


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [15]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [16]:
#!L

model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

In [17]:
#!L
user_item_t_csr

<3953x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 575281 stored elements in Compressed Sparse Row format>

В качестве loss здесь всеми любимый RMSE

In [18]:
#!L

model.fit(user_item_t_csr)

In [19]:
#!L
model

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [20]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [22]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '2252    Pleasantville (1998)',
 '2692    Iron Giant, The (1999)',
 '1526    Hercules (1997)',
 '3817    Went to Coney Island on a Mission From God... ...']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [23]:
get_similars(3, model)

['2    Grumpier Old Men (1995)',
 '3381    Grumpy Old Men (1993)',
 '819    First Wives Club, The (1996)',
 '516    Robin Hood: Men in Tights (1993)',
 '184    Nine Months (1995)',
 '234    Forget Paris (1995)',
 '1619    Bean (1997)',
 '482    Life with Mikey (1993)',
 "1420    McHale's Navy (1997)",
 '1369    My Fellow Americans (1996)']

In [26]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [27]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [28]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [29]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '1892    Rain Man (1988)',
 '1179    Princess Bride, The (1987)',
 '1884    French Connection, The (1971)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [30]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [31]:
#!L

device = torch.device('cuda')
device

device(type='cuda')

In [32]:
user_ids = list(set(ratings['user_id']))
user_ids.sort()

user_ids[0], user_ids[-1]

N_USERS = user_ids[-1] + 1
N_USERS

6041

In [33]:
movie_ids = list(set(ratings['movie_id']))
movie_ids.sort()

movie_ids[0], movie_ids[-1]
N_MOVIES = movie_ids[-1] + 1
N_MOVIES

3953

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [34]:
#!L
ratings_sorted = ratings.sort_values(by=['timestamp'])

n = len(ratings_sorted)
train_size = int(n * 0.9)
ratings_train = ratings_sorted.loc[:train_size]
ratings_test  = ratings_sorted.loc[train_size:]

In [35]:
#!L
ratings_test

,user_id,movie_id,rating,timestamp
900188,5443,1954,3,959980864
900028,5442,3728,4,959980872
900043,5443,590,4,959980892
900200,5443,1962,5,959980892
899991,5442,318,5,959980898
...,...,...,...,...
825793,4958,2399,1,1046454338
825438,4958,1407,5,1046454443
825724,4958,3264,4,1046454548
825731,4958,2634,3,1046454548


In [51]:
#!L

class SvdModel(nn.Module):
    def __init__(self, n_u, n_v, alpha=1e-4):
        super(SvdModel, self).__init__()
        dim = 64
        self.u = nn.Embedding(n_u, dim)
        self.v = nn.Embedding(n_v, dim)
        self.u_bias = nn.Embedding(n_u, 1)
        self.v_bias = nn.Embedding(n_v, 1)
        
        scale = 1. / np.sqrt(dim)
        nn.init.uniform_(self.u.weight, 0, scale)
        nn.init.uniform_(self.v.weight, 0, scale)
        nn.init.uniform_(self.u_bias.weight, 0, scale)
        nn.init.uniform_(self.v_bias.weight, 0, scale)
        self.alpha = alpha
    
    def forward(self, u_idx, v_idx):
        u_embedding = self.u(u_idx) + self.u_bias(u_idx)
        v_embedding = self.v(v_idx) + self.v_bias(v_idx)
        return (u_embedding * v_embedding).sum(dim=1)
    
    def reg(self, u_idx, v_idx):
        u_norm = (self.u(u_idx) ** 2).sum(dim=1).mean()
        v_norm = (self.v(v_idx) ** 2).sum(dim=1).mean()
        return self.alpha * (v_norm + u_norm)

class SvdRecomender:
    def __init__(self, n_u, n_v, lr=0.01):
        self.model = SvdModel(n_u, n_v, alpha=1e-3).to(device)
        self.lr = lr
    
    def fit(self, ratings, n_epoch=2000, batch_size=10000):
        n = len(ratings)
        print(type(ratings))
        user_ids  = torch.tensor(ratings['user_id'].values, dtype=torch.long).to(device)
        movie_ids = torch.tensor(ratings['movie_id'].values, dtype=torch.long).to(device)
        r         = torch.tensor(ratings['rating'].values, dtype=torch.float).to(device)
        
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr)
        perm = torch.randperm(n).to(device)
        
        for epoch in range(n_epoch):
            optimizer.zero_grad()
            losses = []
            for l in range(0, n - batch_size, batch_size):
                batch_idx = perm[l: l + batch_size]

                u_idx = user_ids[batch_idx]
                v_idx = movie_ids[batch_idx]
                cur_r = r[batch_idx]
                pred = self.model.forward(u_idx, v_idx)
                loss = ((pred - cur_r) ** 2).mean() + self.model.reg(u_idx, v_idx)
                loss.backward()
                losses.append(loss.detach().item())
            if epoch % 50 == 0:
                print(f'epoch {epoch}/{n_epoch}')
                print(f'average loss: {np.average(losses)}')
                perm = torch.randperm(n).to(device)
        
            optimizer.step()
        return self
        
    def predict(self, df):
        user_ids  = torch.tensor(df['user_id'].values,  dtype=torch.long).to(device)
        movie_ids = torch.tensor(df['movie_id'].values, dtype=torch.long).to(device)
        
        pred = self.model.forward(user_ids, movie_ids)
        return pred.detach().cpu().numpy()

In [52]:
#!L
model = SvdRecomender(N_USERS, N_MOVIES).fit(ratings, n_epoch=300)

<class 'pandas.core.frame.DataFrame'>
epoch 0/300
average loss: 8.033027086257935
epoch 50/300
average loss: 0.8728708279132843
epoch 100/300
average loss: 0.8310353976488113
epoch 150/300
average loss: 0.8223923051357269
epoch 200/300
average loss: 0.8193404120206833
epoch 250/300
average loss: 0.8178995662927627


In [58]:
#!L

def recommend(user_id, model, k=10):
    movie_ids = np.arange(N_MOVIES)
    df = pd.DataFrame.from_dict({
        'user_id' : np.repeat(user_id, len(movie_ids)),
        'movie_id': movie_ids
    })
    r = model.predict(df)
    
    movies_seen = ratings[ratings['user_id'] == user_id]['movie_id']
    movie_ids[movies_seen] = 0
    movie_ids[0] = 0
    
    return r.argsort()[::-1][:k]

movie_ids = recommend(4, model)

[movie_info[movie_info["movie_id"] == movie_id]["name"].to_string() for movie_id in movie_ids]

['2836    Sanjuro (1962)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '315    Shawshank Redemption, The (1994)',
 '1162    Paths of Glory (1957)',
 '910    Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)',
 '49    Usual Suspects, The (1995)',
 '847    Godfather, The (1972)',
 '735    Close Shave, A (1995)',
 '1132    Wrong Trousers, The (1993)',
 '1194    Third Man, The (1949)']

In [61]:
# TODO: figure out why it works soooo bad

In [54]:
#!L

recomender = SvdRecomender(N_USERS, N_MOVIES).fit(ratings_train, n_epoch=300)

<class 'pandas.core.frame.DataFrame'>
epoch 0/300
average loss: 8.367157816886902
epoch 50/300
average loss: 1.2411806136369705
epoch 100/300
average loss: 0.9616048112511635
epoch 150/300
average loss: 0.8883223757147789
epoch 200/300
average loss: 0.8583320528268814
epoch 250/300
average loss: 0.8433878794312477


In [55]:
#!L

train_rmse = np.sqrt(np.average((recomender.predict(ratings_train) - ratings_train['rating'].values) ** 2))
train_rmse

0.9142395860581849

In [56]:
#!L

test_rmse = np.sqrt(np.average((recomender.predict(ratings_test) - ratings_test['rating'].values) ** 2))
test_rmse

2.334543001042999

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [239]:
from scipy.linalg import inv

class AlsRecomender:
    def __init__(self, n_u, n_v, alpha=1e-4):
        self.dim = 64
        self.U = None
        self.V = None
        self.n_u = n_u
        self.n_v = n_v
        self.alpha = 1e-4
    
    def fit(self, ratings, n_iters=100):
        U = np.random.uniform(0, scale, size=[self.n_u, self.dim])
        V = np.random.uniform(0, scale, size=[self.n_v, self.dim])
        for i in range(n_iters):
            # TODO fix the algorithm
            
            U_update = np.zeros_like(U)
            V_inv = inv(np.dot(V.T, V) + self.alpha * np.eye(self.dim))
            for user_id, movie_id in zip(ratings['user_id'], ratings['movie_id']):
                U_update[user_id] += V_inv.dot(V[movie_id])
            U = U_update
             
            V_update = np.zeros_like(V)
            U_inv = inv(np.dot(U.T, U) + self.alpha * np.eye(self.dim))
            for user_id, movie_id in zip(ratings['user_id'], ratings['movie_id']):
                  V_update[movie_id] += U_inv.dot(U[user_id])
            V = V_update
        self.V = V
        self.U = U
        return self
    
    def predict(self, df):
        user_ids  = ratings['user_id'].values
        movie_ids = ratings['movie_id'].values
        return self.U[user_ids] * self.V[movie_ids]

In [57]:
# TODO

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [ ]:
# TODO

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [ ]:
# TODO